In [7]:
import pandas as pd

merged_merchant_with_categories = pd.read_parquet('../data/curated/merged_merchant_info_with_categories.parquet')

In [8]:
merged_merchant_with_categories.head()

,merchant_abn,category_label,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions
0,10023283211,Furniture and Home Furnishing,0.18,0.0,216.736529,155.285714
21,10142254217,"Media and Technology, Services",4.22,0.0,37.716001,144.571429
42,10165489824,Retail Goods,4.40,0.0,11236.094771,0.454545
53,10187291046,"Automotives, Retail Goods, Services",3.29,0.0,119.526004,16.800000
73,10192359162,Retail Goods,6.33,0.0,462.329450,19.250000


In [9]:
consumer_tbl = pd.read_csv('../data/raw/tbl_consumer.csv', delimiter='|')
userdf = pd.read_parquet('../data/raw/consumer_user_details.parquet')
merged = consumer_tbl.merge(userdf, on='consumer_id', how='left')

In [10]:
test_df = pd.read_parquet('../test/')

In [11]:
merged_with_test = merged.merge(test_df, on='user_id', how='left')
merged_with_test_dropped = merged_with_test.dropna()

In [12]:
# Dropped na since no one has bought from these merchants
merged_with_abn = merged_with_test_dropped.merge(merged_merchant_with_categories, on='merchant_abn', how='left')
merged_with_abn_dropped = merged_with_abn.dropna()

In [13]:
columns_to_drop = ['name', 'address', 'state', 'gender', 'dollar_value', 'order_id', 'order_datetime', 'category_label']

cleaned_merged_with_abn = merged_with_abn_dropped.drop(columns=columns_to_drop)

In [14]:
census_data = pd.read_csv('../data/curated/2021Census_G02_AUST_POA_curated.csv')
census_data = census_data.rename(columns={'POA_CODE_2021': 'postcode'})
census_data.head()
merged_with_census = cleaned_merged_with_abn.merge(census_data, on='postcode', how='left')

print(merged_with_census.head())
merged_with_census.isnull().sum()

   postcode  consumer_id  user_id  merchant_abn  take_rate_value  \
0      6935      1195503        1  2.800049e+10             4.24   
1      6935      1195503        1  8.369064e+10             3.15   
2      6935      1195503        1  9.668077e+10             5.91   
3      6935      1195503        1  4.989171e+10             5.80   
4      6935      1195503        1  6.240384e+10             6.46   

   fraud_probability  average_monthly_revenue  average_monthly_transactions  \
0           0.000000               245.258150                    180.523810   
1           0.000000                88.512653                   1707.238095   
2          29.555245               316.660874                   1482.571429   
3           0.000000                28.889602                  11786.952381   
4           0.000000                34.467794                    184.571429   

   Median_age_persons  Median_mortgage_repay_monthly  Median_rent_weekly  \
0                 NaN                   

postcode                               0
consumer_id                            0
user_id                                0
merchant_abn                           0
take_rate_value                        0
fraud_probability                      0
average_monthly_revenue                0
average_monthly_transactions           0
Median_age_persons               2244225
Median_mortgage_repay_monthly    2244225
Median_rent_weekly               2244225
Median_tot_hhd_inc_weekly        2244225
dtype: int64

In [15]:
merged_with_census_dropped = merged_with_census.dropna()
merged_with_census_dropped.head()

,postcode,consumer_id,user_id,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
553,2782,179208,2,8.077982e+10,4.71,0.0,34.567393,1735.142857,50.0,2080.0,430.0,1528.0
554,2782,179208,2,2.864764e+10,3.32,0.0,263.829390,23.476190,50.0,2080.0,430.0,1528.0
555,2782,179208,2,3.581349e+10,5.66,0.0,45.885892,182.761905,50.0,2080.0,430.0,1528.0
556,2782,179208,2,3.809009e+10,3.24,0.0,172.158074,2467.428571,50.0,2080.0,430.0,1528.0
557,2782,179208,2,3.897149e+10,3.61,0.0,166.642856,269.761905,50.0,2080.0,430.0,1528.0


In [16]:
aggregated_df = merged_with_census_dropped.groupby('merchant_abn').mean()
aggregated_df = aggregated_df.reset_index()
aggregated_df = aggregated_df.drop(columns=['consumer_id', 'postcode', 'user_id'])

aggregated_df.head()

,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053


In [17]:
merchant_categories = pd.read_parquet('../data/curated/merchant_categories.parquet')
merchant_categories = merchant_categories.drop(columns=['category', 'revenue_rate', 'fraud_probability', 'take_rate_value'])
category_df = aggregated_df.merge(merchant_categories, on='merchant_abn', how='left')



In [45]:
merchant_fraud = pd.read_csv('../data/raw/merchant_fraud_probability.csv')
final_df = category_df.merge(merchant_fraud, on='merchant_abn', how='left')
final_df['fraud_probability_y'] = final_df['fraud_probability_y'].apply(lambda x: 0 if pd.isna(x) else 1)
final_df = final_df.drop(columns = ['order_datetime'])
final_df = final_df.rename(columns={'fraud_probability_x': 'consumer_fraud', 'fraud_probability_y': 'merchant_fraud'})


In [46]:
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0


In [47]:
final_df['Median_age_persons'] = (25 - final_df['Median_age_persons']).abs()
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,18.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,18.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,14.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,18.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,18.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0


In [48]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming final_df is your DataFrame

# Columns to exclude from standardization
exclude_columns = ['merchant_abn', 'consumer_fraud', 'merchant_fraud', 'category_label', 'name']

# Columns to standardize
columns_to_standardize = [col for col in final_df.columns if col not in exclude_columns]

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply the scaler to the columns to be standardized
final_df[columns_to_standardize] = scaler.fit_transform(final_df[columns_to_standardize])

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,-2.427244,0.0,-0.413475,-0.011175,-0.080343,-0.081199,-0.100735,-0.053952,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,-0.090958,0.0,-0.460609,-0.027384,-0.153495,0.052766,-0.025484,0.083467,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,0.013134,0.0,2.487786,-0.245407,-3.533623,1.081669,1.685615,3.174476,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,-0.628766,0.0,-0.439069,-0.220679,0.340493,0.032559,-0.275510,-0.081271,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0
4,1.019236e+10,1.129232,0.0,-0.348813,-0.216973,-0.151571,0.226256,0.497324,0.690895,Enim Condimentum PC,Retail Goods,0


In [37]:
#final_df['Median_age_persons'] = (1 - final_df['Median_age_persons'])
#final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.011594,0.0,0.004023,0.011255,0.532351,0.154487,0.566743,0.341571,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,0.597101,0.0,0.000571,0.010478,0.534692,0.156682,0.569549,0.346294,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,0.623188,0.0,0.216470,0.000024,0.642857,0.173542,0.633333,0.452535,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,0.462319,0.0,0.002148,0.001210,0.518885,0.156351,0.560228,0.340632,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0
4,1.019236e+10,0.902899,0.0,0.008758,0.001387,0.534631,0.159525,0.589037,0.367172,Enim Condimentum PC,Retail Goods,0


In [49]:
import pandas as pd

# Assuming final_df is your DataFrame

# Define the weights for each column
weights = {
    'merchant_fraud': -1.5,
    'average_monthly_revenue': 1.5,
    'take_rate_value': 0.5,
    'consumer_fraud': -0.6,
    'average_monthly_transactions': 0.9,
    'Median_tot_hhd_inc_weekly': 0.2,
    'Median_age_persons': -0.15,
    'Median_mortgage_repay_monthly': -0.1,
    'Median_rent_weekly': -0.1
}

# Calculate the score for each row
final_df['score'] = (
    final_df['merchant_fraud'] * weights['merchant_fraud'] +
    final_df['average_monthly_revenue'] * weights['average_monthly_revenue'] +
    final_df['take_rate_value'] * weights['take_rate_value'] +
    final_df['consumer_fraud'] * weights['consumer_fraud'] +
    final_df['average_monthly_transactions'] * weights['average_monthly_transactions'] +
    final_df['Median_tot_hhd_inc_weekly'] * weights['Median_tot_hhd_inc_weekly'] +
    final_df['Median_age_persons'] * weights['Median_age_persons'] +
    final_df['Median_mortgage_repay_monthly'] * weights['Median_mortgage_repay_monthly'] +
    final_df['Median_rent_weekly'] * weights['Median_rent_weekly']
)

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,score
0,1.002328e+10,-2.427244,0.0,-0.413475,-0.011175,-0.080343,-0.081199,-0.100735,-0.053952,Felis Limited,Furniture and Home Furnishing,0,-1.824437
1,1.014225e+10,-0.090958,0.0,-0.460609,-0.027384,-0.153495,0.052766,-0.025484,0.083467,Arcu Ac Orci Corporation,"Media and Technology, Services",0,-0.724048
2,1.016549e+10,0.013134,0.0,2.487786,-0.245407,-3.533623,1.081669,1.685615,3.174476,Nunc Sed Company,Retail Goods,0,4.405591
3,1.018729e+10,-0.628766,0.0,-0.439069,-0.220679,0.340493,0.032559,-0.275510,-0.081271,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0,-1.214631
4,1.019236e+10,1.129232,0.0,-0.348813,-0.216973,-0.151571,0.226256,0.497324,0.690895,Enim Condimentum PC,Retail Goods,0,-0.065322


In [50]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Sort the DataFrame by the 'score' column in descending order
sorted_df = final_df.sort_values(by='score', ascending=False)

# Select the top 100 merchants
top_100_merchants = sorted_df.head(100)

# Display the top 100 merchants
top_100_merchants.head(30)

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,score
762,2.485245e+10,-0.831167,0.000000,-0.462626,20.610117,-0.053077,-0.010563,0.005864,0.011693,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0,17.450353
3728,8.657848e+10,1.187061,0.000000,-0.461314,19.397053,-0.026657,0.018087,0.055301,0.053075,Leo In Consulting,"Automotives, Retail Goods, Services",0,17.366182
1580,3.915015e+10,1.510902,0.000000,9.558071,-0.244582,-6.658601,-0.915618,-2.529919,2.129340,Aliquam Eu Institute,"Automotives, Services",0,16.641646
971,2.906845e+10,-0.634549,0.000000,9.258146,-0.244582,16.555520,-9.508990,-15.304266,14.394781,Phasellus LLP,"Automotives, Services",0,16.226775
2683,6.420342e+10,-0.877430,0.000000,-0.462917,18.540222,-0.034216,0.024828,0.063371,0.060682,Pede Nonummy Corp.,Furniture and Home Furnishing,0,15.561559
2068,4.989171e+10,0.822739,0.000000,-0.462932,17.585418,-0.041761,0.004607,0.029068,0.028975,Non Vestibulum Industries,Furniture and Home Furnishing,0,15.552539
1939,4.704774e+10,-1.640772,0.000000,10.296940,-0.244582,-14.694258,1.934598,3.346280,-3.909224,Aenean Gravida Institute,"Automotives, Services",0,15.299107
1870,4.562922e+10,1.505119,0.000000,-0.460905,16.197155,-0.054906,-0.008703,0.022705,0.035794,Lacus Consulting,"Automotives, Retail Goods",0,14.652636
1927,4.680414e+10,-0.836950,0.000000,-0.462609,16.642284,-0.024709,0.016383,0.025012,0.032845,Suspendisse Dui Corporation,Services,0,13.871803
3875,8.972601e+10,0.944179,0.000000,-0.459657,15.311653,-0.061268,0.004655,0.037643,0.046349,Est Nunc Consulting,Furniture and Home Furnishing,0,13.577322


In [51]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Define the category labels
categories = ['Retail Goods', 'Media and Technology', 'Furniture and Home Furnishing', 'Services', 'Automotives']

# Initialize variables to store the top 10 merchants for each category
top_10_retail_goods = None
top_10_media_technology = None
top_10_furniture_home_furnishing = None
top_10_services = None
top_10_automotives = None

# Loop through each category and get the top 10 merchants
for category in categories:
    # Filter and sort the DataFrame by the current category and score
    top_10_merchants = final_df[final_df['category_label'] == category].sort_values(by='score', ascending=False).head(10)
    
    # Store the result in the corresponding variable
    if category == 'Retail Goods':
        top_10_retail_goods = top_10_merchants
    elif category == 'Media and Technology':
        top_10_media_technology = top_10_merchants
    elif category == 'Furniture and Home Furnishing':
        top_10_furniture_home_furnishing = top_10_merchants
    elif category == 'Services':
        top_10_services = top_10_merchants
    elif category == 'Automotives':
        top_10_automotives = top_10_merchants

# Display the top 10 merchants for each category
print("Top 10 merchants in 'Retail Goods':")
print(top_10_retail_goods)
print("\n")

print("Top 10 merchants in 'Media and Technology':")
print(top_10_media_technology)
print("\n")

print("Top 10 merchants in 'Furniture and Home Furnishing':")
print(top_10_furniture_home_furnishing)
print("\n")

print("Top 10 merchants in 'Services':")
print(top_10_services)
print("\n")

print("Top 10 merchants in 'Automotives':")
print(top_10_automotives)
print("\n")

Top 10 merchants in 'Retail Goods':
      merchant_abn  take_rate_value  consumer_fraud  average_monthly_revenue  \
2820  6.726425e+10         1.169712       18.210891                13.187938   
2679  6.414355e+10         0.869002        0.000000                 4.998503   
2608  6.268859e+10        -2.022442        0.000000                 5.088139   
2690  6.440360e+10         1.117666        0.000000                -0.449969   
1571  3.891866e+10        -0.455280        0.000000                 4.152779   
1917  4.653701e+10        -1.270667        0.000000                 3.227528   
4288  9.759036e+10         1.221758        0.000000                 3.643225   
1810  4.434579e+10         0.978877        0.000000                 3.359250   
968   2.902701e+10        -1.432588        0.000000                 4.553847   
2911  6.918432e+10        -1.490417        0.000000                 3.951799   

      average_monthly_transactions  Median_age_persons  \
2820                     